In [70]:
import jutil
import numpy as np

In [2]:
# 输入学生学号,要预测的课程X,返回该门课是否挂科

In [103]:
# 1.找出X的前置课程,确定学生的专业和年级
sql = "select grade,speciality_code,course_code,course_name,term_order,mark,pmark from view_stu_course_mark where student_id='2015111848'"
stu = jutil.load_pd_df(sql)
stu.head()

,course_code,course_name,grade,mark,pmark,speciality_code,term_order
0,0471004,面向对象程序设计,2015,5,74.0,0408,2
1,0471005,面向对象程序设计实验,2015,9,85.0,0408,2
2,0471054,程序语言综合课程设计,2015,6,77.0,0408,2
3,0471078,高级#语言程序设计,2015,8,84.0,0408,1
4,0471108,操作系统,2015,4,69.0,0408,3


,course_code,course_name,grade,mark,pmark,speciality_code,term_order


In [112]:
grade = stu.iloc[0]['grade']
term_order = stu['term_order'].max()
speciality_code = stu.iloc[0]['speciality_code']

In [113]:
term_order

3

In [119]:
pre_course = stu[stu['term_order']<=term_order]

In [120]:
course_codes = list(pre_course['course_code'].values)
course_codes.append(course_code)
# course_codes
course_names = list(pre_course['course_code'].values)
course_names.append(course_name)

In [20]:
# 找前一个年级的成绩数据来预测

In [76]:
pre_grade = str(int(grade) - 1)
sql = "select student_id,course_name,course_code,pmark,mark from view_stu_course_mark where speciality_code='%s' and grade='%s'" \
    %(speciality_code,pre_grade)
df = jutil.load_pd_df(sql)
df = df.drop_duplicates(subset=['student_id','course_code'])
df.head()

,course_code,course_name,mark,pmark,student_id
0,3143300,数字电子技术实验B(电类),5,74.0,20112807
1,3143328,数据库技术与应用,7,81.0,20112807
2,3232100,数据结构A,5,72.0,20112807
3,6111120,大学物理实验Ⅱ,9,86.7,20112807
4,7044900,信息检索,7,80.0,20112807


In [118]:
course_name = '计算机图形学'
a = df[df['course_name']==course_name].iloc[0]
a['course_code']
# course_code = a['course_code'].values[0]

'3232400'

In [25]:
# 选出需要的课程


In [77]:
stu_ids = df[df['course_code']==course_code]['student_id'].values
d1 = df[df['student_id'].isin(stu_ids)]
dd = d1[d1['course_code'].isin(course_codes)]

In [78]:
now_names = set(dd['course_name'].values)
len(now_names)

61

In [79]:

gd = dd.groupby(['student_id'],as_index=False)
d_arr = dict() # key存的是课程名,value是存放成绩的数组,这样竖着看就是一个学生的成绩列表了,也就是转置操作
d_arr2 = dict() 
for name ,g in gd:
    d_m = dict()
    d_m2 = dict()
    for i,row in g.iterrows():
        d_m[row['course_name']] = row['pmark']
        d_m2[row['course_name']] = row['mark']
    for c in now_names:
        d_arr[c] = d_arr.get(c,[])
        d_arr[c].append(d_m.get(c,60))
        d_arr2[c] = d_arr2.get(c,[])
        d_arr2[c].append(d_m2.get(c,1))
courses = d_arr.keys()
mt_old = [arr for arr in d_arr.values()]
mt2_old = [arr for arr in d_arr2.values()]
mt = np.transpose(mt_old)
mt

array([[85. , 64. , 62. , ...,  4. , 90. , 72. ],
       [85. , 60. , 74. , ..., 80. , 75. , 84. ],
       [85. , 60. , 66. , ..., 84.6, 88. , 92. ],
       ...,
       [85. , 60. , 25. , ..., 80.5, 88. , 80. ],
       [93. , 60. , 68. , ..., 74. , 90. , 81. ],
       [60. , 60. , 65. , ..., 77.1, 86. , 82. ]])

In [80]:
# 要将mt矩阵进行拆分成数据和标签(预测的那门课)
y = d_arr[course_name]
X = np.transpose([v for k,v in d_arr.items() if not k==course_name])
course_old = [k for k in d_arr.keys() if not k==course_name]

In [68]:
X.shape

(116, 60)

In [65]:
len(y)

116

In [81]:
label = []
for m in y:
    if m>65:
        label.append(1)
    else:
        label.append(0)

In [71]:
# 对X做特征选取和降维操作
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [135]:
sel = SelectKBest(chi2, k=20)
X_new = sel.fit_transform(X, label)
indice = sel.get_support()

In [136]:
X_new

array([[61., 64., 76., ..., 85., 81., 75.],
       [60., 60., 46., ..., 65., 83., 83.],
       [75., 74., 76., ..., 68., 81., 72.],
       ...,
       [83., 60., 74., ..., 64., 71., 82.],
       [87., 60., 71., ..., 85., 89., 86.],
       [68., 60., 62., ..., 63., 79., 92.]])

In [74]:
# 进行交叉验证下

In [95]:
from sklearn.model_selection import train_test_split

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.4)

In [82]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score

In [97]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)

In [98]:
y_pred = clf.predict(X_test)

In [99]:
accuracy_score(y_test,y_pred)

0.723404255319149

In [85]:
import graphviz

In [100]:
dot_data = tree.export_graphviz(clf, out_file=None,class_names=True,feature_names=course_old) 
graph = graphviz.Source(dot_data)
graph.render("jimo") 

'jimo.pdf'

In [83]:
clf2 = DecisionTreeClassifier()
clf2 = clf2.fit(X,label)

In [84]:
y_pred2 = clf2.predict(X)
accuracy_score(label,y_pred2)

1.0

In [87]:
course_choose = [course_old[i] for i in range(len(course_old)) if indice[i]]

NameError: name 'indice' is not defined

In [96]:
# 进行预测
course_choose

['马克思主义基本原理',
 '离散数学A',
 '线性代数B',
 '计算方法C',
 '高级语言程序设计',
 '软件项目管理',
 '算法设计与分析',
 '计算机组成实验C',
 '信息检索',
 '数据结构实验',
 '面向对象程序设计',
 '高等数学Ⅰ',
 '英语Ⅱ',
 '面向对象程序设计实验',
 '高等数学Ⅱ',
 '毛泽东思想和中国特色社会主义理论体系概论Ⅰ',
 '概率论与数理统计B',
 '大学物理BⅡ',
 '微机与接口技术实验',
 '操作系统']

In [88]:
xx = []
for c in course_old:
    v = stu[stu['course_name']==c]['pmark'].values
    if len(v)>0:
        xx.append(v[0])
    else: xx.append(60)
x_n = [xx]

In [90]:
clf2.predict(x_n)

array([1])

In [92]:
x_1 = [[50 for i in range(60)]]
clf2.predict(x_1)

array([1])

## 下面是找出学生未修课程

In [54]:
from datetime import datetime
import numpy as np

In [5]:
speciality_code = stu.iloc[0]['speciality_code']
grade = stu.iloc[0]['grade']

In [47]:
sql = "select t.Course_name as course_name,t.Course_year as year,t.Course_term as term,t.Course_code as course_code,c.type_name as course_type from \
train_plan_course t,train_plan_credit c where t.Course_big_type_id=c.sid \
and t.Grade='%s' and c.type_name like '%%专业%%'  and t.Speciality_code='%s'" %(2015,speciality_code)
df2 = jutil.load_pd_df(sql)
df2.head()

,course_code,course_name,course_type,term,year
0,3273522,数字电子技术实验B,学科与专业基础课程,1,2
1,3273521,离散数学,学科与专业基础课程,2,1
2,3273573,计算机组成原理,学科与专业基础课程,2,2
3,0474012,操作系统实验,学科与专业基础课程,2,2
4,0474011,数据结构实验,学科与专业基础课程,1,2


In [69]:
# 计算当前学期
today = datetime.today()
year = str(today.year - int(grade))
term = '1' if today.month>2 and today.month<7 else '2'
df3 = df2[df2['year']==year]
df3 = df3[df3['term']>=term]
df3

,course_code,course_name,course_type,term,year
38,0473066,移动计算,专业（专业方向）课程,1,4
39,0673139,数据挖掘,专业（专业方向）课程,1,4
40,0473003,嵌入式系统设计与应用,专业（专业方向）课程,1,4
41,0473004,嵌入式系统设计与应用实验,专业（专业方向）课程,1,4
42,0473042,软件设计模式,专业（专业方向）课程,1,4
46,3273101,云计算与并行技术,专业（专业方向）课程,1,4
48,3273528,人工智能,专业（专业方向）课程,1,4


In [61]:
# 找出所有专业课程
course_all = set(df3['course_name'].values)

In [108]:
# 找出学生已考试的课程
course_did = set(stu['course_name'].values)

In [109]:
# 求差集,剩下的就是没修的课程
course_all - course_did

{'云计算与并行技术', '人工智能', '嵌入式系统设计与应用', '嵌入式系统设计与应用实验', '数据挖掘', '移动计算', '软件设计模式'}

In [65]:
df4 = df2[df2['year'] > year]
df4

,course_code,course_name,course_type,term,year


In [68]:
df4.append(df3)

,course_code,course_name,course_type,term,year
38,0473066,移动计算,专业（专业方向）课程,1,4
39,0673139,数据挖掘,专业（专业方向）课程,1,4
40,0473003,嵌入式系统设计与应用,专业（专业方向）课程,1,4
41,0473004,嵌入式系统设计与应用实验,专业（专业方向）课程,1,4
42,0473042,软件设计模式,专业（专业方向）课程,1,4
46,3273101,云计算与并行技术,专业（专业方向）课程,1,4
48,3273528,人工智能,专业（专业方向）课程,1,4
